# Model

In [ ]:
import torch
import sklearn.model_selection
import sklearn.preprocessing

### Define the Model

In [ ]:
class LSTM(torch.nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = torch.nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            batch_first=True
                           )
        self.fully_connected1 = torch.nn.Linear(hidden_size, 128)
        self.fully_connected2 = torch.nn.Linear(128, num_classes)
        
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        # Initialize states
        initial_hidden_state   = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        initial_internal_state = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Propogate forward
        output, (hidden_state, internal_state) = self.lstm(x, (initial_hidden_state, initial_internal_state))
        hidden_state = hidden_state.view(-1, self.hidden_size)  # Reshape for next dense layer
        output = self.relu(hidden_state)
        output = self.fully_connected1(output)
        output = self.relu(output)
        output = self.fully_connected2(output)
        
        return output
        

### Run Model

In [ ]:
# Model parameters
N_CLASSES = 1
INPUT_SIZE = X_train_tensors.shape[2]  # Number of features
HIDDEN_SIZE = 2 # Number of features in hidden state
N_LAYERS = 1  # Number of stacked LSTM layers
BIAS = X_train_tensors.shape[1]

# Training parameters
N_EPOCHS = 100000
LEARNING_RATE = 0.01
loss_func = torch.nn.MSELoss()


lstm = LSTM(N_CLASSES, INPUT_SIZE, HIDDEN_SIZE, N_LAYERS, BIAS)

# Training loop
optimizer = torch.optim.Adam(lstm.parameters(), lr=LEARNING_RATE)
for epoch_i in range(1, N_EPOCHS+1):
    # Forward propogation
    outputs = lstm.forward(X_train_tensors)
    optimizer.zero_grad()  # Manually set gradient to 0 here
    
    # Backward propogation
    np_outputs = outputs.detach().numpy()
    #if np.all(np_outputs == np_outputs[0]):
        #print('Epoch {}: All outputs the same'.format(epoch_i))
    loss = loss_func(outputs, y_train_tensors)
    loss.backward()
    
    optimizer.step()
    if epoch_i % 1000 == 0:
        print('Epoch: {}, loss: {:.2f}'.format(epoch_i, loss.item()))

### Evaluate

In [ ]:
print(X_val_tensors)

val_predictions = lstm(X_val_tensors).data.numpy()  # Forward pass

plt.figure(figsize=(10, 6))

#plt.plot(y_val, label='Actual')
plt.plot(val_predictions, label='Predicted')
plt.legend()
plt.show()

y_val.head()

In [ ]:
print(X_val_tensors.mean())